In [ ]:

import pandas as pd

# CSV 파일 로드
df = pd.read_csv("ts2000_y_processed_v1.csv")
df.head()


In [ ]:

from scipy.stats.mstats import winsorize

# 수치형 컬럼 중 is_defaulted 제외
numeric_cols = df.select_dtypes(include='number').columns.tolist()
numeric_cols = [col for col in numeric_cols if col != 'is_defaulted']

df_winsorized = df.copy()

# 각 컬럼에 대해 상하단 1% 윈저라이징 적용
for col in numeric_cols:
    col_data = df_winsorized[col].dropna()
    winsorized_data = winsorize(col_data, limits=[0.01, 0.01])
    df_winsorized.loc[col_data.index, col] = winsorized_data


In [ ]:

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows 기준
plt.rcParams['axes.unicode_minus'] = False

# 히스토그램 (예시로 상위 5개 컬럼만)
for col in numeric_cols[:5]:
    plt.figure(figsize=(8, 4))
    df_winsorized[col].hist(bins=50)
    plt.title(f'{col} 분포 (윈저라이징 적용)')
    plt.xlabel(col)
    plt.ylabel('빈도수')
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:

df_winsorized.to_csv("ts2000_y_winsorized.csv", index=False)
print("윈저라이징된 데이터가 'ts2000_y_winsorized.csv'로 저장되었습니다.")
